# Train GLoVE on danish WIKI-data

In [0]:
#! conda install llvm gcc libgcc
#! pip install glove_python
#! pip install Gensim
! pip install https://github.com/JonathanRaiman/glove/archive/master.zip
#! pip install sklearn
#! pip install wheel 

# https://github.com/sorenlind/lemmy
#! pip install Lemmy 

     | 30kB 41.6MB/s
  Created wheel for glove: filename=glove-1.0.1-cp36-none-any.whl size=5000 sha256=2066cff61908576ea0cb41bc7a279c16eb3bd9837b4a1b2978bef334992a38a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-w9iju3pc/wheels/d8/f8/8d/8754c24e9f908071d7d0d903591b06d9915b56a491abaae45a
Successfully built glove


In [0]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11JZMOg6szubeTIILAxpsv-WsMcmDfYsD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11JZMOg6szubeTIILAxpsv-WsMcmDfYsD" -O danish_words_thina.json && rm -rf /tmp/cookies.txt

--2020-04-30 18:17:43--  https://docs.google.com/uc?export=download&confirm=Bo7C&id=11JZMOg6szubeTIILAxpsv-WsMcmDfYsD
Resolving docs.google.com (docs.google.com)... 172.217.218.101, 172.217.218.113, 172.217.218.138, ...
Connecting to docs.google.com (docs.google.com)|172.217.218.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-6c-docs.googleusercontent.com/docs/securesc/tvng1bs7bq02mit2td6lqtbke4hah1vn/rhcg9cvuibhm8grain35lumgg29695v7/1588270650000/11375003261469243618/09949696177093222139Z/11JZMOg6szubeTIILAxpsv-WsMcmDfYsD?e=download [following]
--2020-04-30 18:17:44--  https://doc-00-6c-docs.googleusercontent.com/docs/securesc/tvng1bs7bq02mit2td6lqtbke4hah1vn/rhcg9cvuibhm8grain35lumgg29695v7/1588270650000/11375003261469243618/09949696177093222139Z/11JZMOg6szubeTIILAxpsv-WsMcmDfYsD?e=download
Resolving doc-00-6c-docs.googleusercontent.com (doc-00-6c-docs.googleusercontent.com)... 108.177.119.132, 2a00:1450:4013:c00::84
Conne

In [0]:
import json
aaoW = ['Ã¦', 'Ã¸', 'Ã¥', 'Ã†', 'Ã˜', 'Ã…']
aaoC = 'æ ø å Æ Ø Å'.split(' ')
with open('danish_words_thina.json','r', encoding='latin1') as file: 
    text = json.loads(file.read())

print('Loaded in mem!')
for i in range(len(text)):
    if i % 100000 == 0:
        print(i * 100.0 / 14600000)
    for w, c in zip(aaoW, aaoC): # bad fix for bad encoding
        text[i] = [word.replace(w, c) for word in text[i]]
    text[i] = [word.lower() for word in text[i]]

Loaded in mem!
0.0
0.684931506849315
1.36986301369863
2.0547945205479454
2.73972602739726
3.4246575342465753
4.109589041095891
4.794520547945205
5.47945205479452
6.164383561643835
6.8493150684931505
7.534246575342466
8.219178082191782
8.904109589041095
9.58904109589041
10.273972602739725
10.95890410958904
11.643835616438356
12.32876712328767
13.013698630136986
13.698630136986301
14.383561643835616
15.068493150684931
15.753424657534246
16.438356164383563
17.123287671232877
17.80821917808219
18.493150684931507
19.17808219178082
19.863013698630137
20.54794520547945
21.232876712328768
21.91780821917808
22.602739726027398
23.28767123287671
23.972602739726028
24.65753424657534
25.34246575342466
26.027397260273972
26.71232876712329
27.397260273972602
28.08219178082192
28.767123287671232
29.45205479452055
30.136986301369863
30.82191780821918
31.506849315068493
32.19178082191781
32.87671232876713
33.56164383561644
34.24657534246575
34.93150684931507
35.61643835616438
36.3013698630137
36.9863013

In [0]:
import lemmy

# Create an instance of the standalone lemmatizer.
lemmatizer = lemmy.load("da")

# Find lemma for the words in text file. First argument is an empty POS tag.
for i in range(len(text)): 
    if i % 100000 == 0:
        print(i * 100.0 / 14600000)

    for j in range(len(text[i])): 
        text[i][j] = lemmatizer.lemmatize("", text[i][j])

In [0]:
print(text[0])

s = set()
for line in text:
    for word in line:
        s.add(word)
print(len(s))

['arkæologi', 'studiet', 'tiders', 'menneskelige', 'aktivitet', 'primært', 'studiet', 'menneskets', 'materielle', 'levn']
2003486


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer = CountVectorizer()
corpus = ['Dette er en sætning til at teste ting og sager']
X = vectorizer.fit_transform(corpus)

In [0]:
import numpy as np
n_words = len(s)
m = {}
m_words = {}
count = 0
for word in s:
  m_words[word] = count
  count += 1

def cal_occ(sentence,m, window):
    for i,word in enumerate(sentence):
        for j in range(max(i-window,0),min(i+window,len(sentence))):
            n1 = m_words[word]
            n2 = m_words[sentence[j]]
            m1 = min(n1, n2)
            m2 = max(n1, n2)
            index = m1 * (n_words + 1) + m2
            if not index in  m:
              m[index] = 1
            else:
              m[index] += 1

for i,sentence in enumerate(text):
    cal_occ(sentence, m, 15)
    if i % 100000 == 0:
            print(i * 100.0 / 14600000)

0.0
0.684931506849315
1.36986301369863
2.0547945205479454
2.73972602739726
3.4246575342465753
4.109589041095891
4.794520547945205
5.47945205479452
6.164383561643835
6.8493150684931505
7.534246575342466
8.219178082191782
8.904109589041095
9.58904109589041
10.273972602739725
10.95890410958904
11.643835616438356
12.32876712328767
13.013698630136986
13.698630136986301
14.383561643835616
15.068493150684931
15.753424657534246
16.438356164383563
17.123287671232877
17.80821917808219
18.493150684931507
19.17808219178082
19.863013698630137
20.54794520547945
21.232876712328768
21.91780821917808
22.602739726027398
23.28767123287671
23.972602739726028
24.65753424657534
25.34246575342466
26.027397260273972
26.71232876712329
27.397260273972602
28.08219178082192
28.767123287671232
29.45205479452055
30.136986301369863
30.82191780821918
31.506849315068493
32.19178082191781
32.87671232876713
33.56164383561644
34.24657534246575
34.93150684931507
35.61643835616438
36.3013698630137
36.986301369863014
37.671

In [0]:
del text[:] # more space
s.clear()
print(len(m))

113865170


In [0]:
occ = {}
while True:
  i = 0
  keys_to_delete = []
  for index in m:
    if i % 1000000 == 0:
      print(i * 100.0 / len(m))
    if i == 1000000:
      break
    keys_to_delete.append(index)
    i += 1
    m1 = index // (n_words + 1)
    m2 = index % (n_words + 1)
    if m1 not in occ:
      occ[m1] = {}
    if m2 not in occ:
      occ[m2] = {}
    occ[m1][m2] = m[index]
    occ[m2][m1] = m[index]
  for k in keys_to_delete:
    del m[k]
  if len(m) == 0:
    break

0.0
0.8782316840171582
0.0
0.8860129302955021
0.0
0.893933294876323
0.0
0.9019965422864548
0.0
0.9102065741126146
0.0
0.9185674352963396
0.0
0.9270833207790801
0.0
0.9357585825203852
0.0
0.94459773691385
0.0
0.9536054726273747
0.0
0.9627866588963365
0.0
0.9721463543004887
0.0
0.9816898160578341
0.0
0.9914225098713461
0.0
1.0013501203672912
0.0
1.0114785621670401
0.0
1.0218139916376787
0.0
1.0323628193704713
0.0
1.0431317234403277
0.0
1.0541276635038972
0.0
1.0653578957988357
0.0
1.076829989112172
0.0
1.088551841791617
0.0
1.100531699880163
0.0
1.11277817646147
0.0
1.125300272311413
0.0
1.1381073979598515
0.0
1.1512093972762616
0.0
1.1646165727034605
0.0
1.1783397122753658
0.0
1.1923901185676962
0.0
1.2067796397449013
0.0
1.2215207028825568
0.0
1.236626349762203
0.0
1.252110275355327
0.0
1.267986869235177
0.0
1.2842712601796156
0.0
1.300979364255618
0.0
1.31812793670666
0.0
1.335734627998574
0.0
1.3538180444179577
0.0
1.372397813660491
0.0
1.3914946558951993
0.0
1.4111304608455748
0.0
1

In [0]:
m = ""
np.save('occ.npy', occ) 
np.save('m_words.npy', m_words) 

In [0]:
#! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qnOAYztp94FvNkCm4y3ndah0viJgiI3p' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qnOAYztp94FvNkCm4y3ndah0viJgiI3p" -O occ.npy && rm -rf /tmp/cookies.txt

--2020-05-05 17:04:11--  https://docs.google.com/uc?export=download&confirm=1aSm&id=1qnOAYztp94FvNkCm4y3ndah0viJgiI3p
Resolving docs.google.com (docs.google.com)... 172.217.204.113, 172.217.204.102, 172.217.204.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.204.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-84-docs.googleusercontent.com/docs/securesc/6tqtrn729b12rof009r43ggi4372i91c/f847r8br2olbt2hfv5q1fi79k1ci29bo/1588698225000/11375003261469243618/02106527022495999023Z/1qnOAYztp94FvNkCm4y3ndah0viJgiI3p?e=download [following]
--2020-05-05 17:04:11--  https://doc-04-84-docs.googleusercontent.com/docs/securesc/6tqtrn729b12rof009r43ggi4372i91c/f847r8br2olbt2hfv5q1fi79k1ci29bo/1588698225000/11375003261469243618/02106527022495999023Z/1qnOAYztp94FvNkCm4y3ndah0viJgiI3p?e=download
Resolving doc-04-84-docs.googleusercontent.com (doc-04-84-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400c:c07::84
Conne

In [0]:
import numpy as np
m_words = np.load('/content/drive/My Drive/m_words.npy',allow_pickle='TRUE').item()
#occ = np.load('occ.npy',allow_pickle='TRUE').item()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import glove
model = glove.Glove(occ, d=50, alpha=0.75, x_max=100.0)
print('Loaded')
for epoch in range(25):
    err = model.train(batch_size=200, workers=9)
    print("epoch %d, error %.3f" % (epoch, err), flush=True)


#The trained embeddings are now present under `model.W`.

/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.6/dist-packages/glove/glove_inner.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Loaded
epoch 0, error 0.037
epoch 1, error 0.026
epoch 2, error 0.023
epoch 3, error 0.021
epoch 4, error 0.020
epoch 5, error 0.019
epoch 6, error 0.019
epoch 7, error 0.018
epoch 8, error 0.018
epoch 9, error 0.018
epoch 10, error 0.017
epoch 11, error 0.017
epoch 12, error 0.017
epoch 13, error 0.017
epoch 14, error 0.017
epoch 15, error 0.017
epoch 16, error 0.017
epoch 17, error 0.016
epoch 18, error 0.016
epoch 19, error 0.016
epoch 20, error 0.016
epoch 21, error 0.016
epoch 22, error 0.016
epoch 23, error 0.016
epoch 24, error 0.016


In [0]:
while True:
  keys = []
  i = 0
  for key in occ:
    keys.append(key)
    i += 1
    if i >= 10000:
      break
  for key in keys:
    del occ[key]
  if len(occ) == 0:
    break
  print(len(occ))
import gc
gc.collect()
#del m_words
#del occ

1993486
1983486
1973486
1963486
1953486
1943486
1933486
1923486
1913486
1903486
1893486
1883486
1873486
1863486
1853486
1843486
1833486
1823486
1813486
1803486
1793486
1783486
1773486
1763486
1753486
1743486
1733486
1723486
1713486
1703486
1693486
1683486
1673486
1663486
1653486
1643486
1633486
1623486
1613486
1603486
1593486
1583486
1573486
1563486
1553486
1543486
1533486
1523486
1513486
1503486
1493486
1483486
1473486
1463486
1453486
1443486
1433486
1423486
1413486
1403486
1393486
1383486
1373486
1363486
1353486
1343486
1333486
1323486
1313486
1303486
1293486
1283486
1273486
1263486
1253486
1243486
1233486
1223486
1213486
1203486
1193486
1183486
1173486
1163486
1153486
1143486
1133486
1123486
1113486
1103486
1093486
1083486
1073486
1063486
1053486
1043486
1033486
1023486
1013486
1003486
993486
983486
973486
963486
953486
943486
933486
923486
913486
903486
893486
883486
873486
863486
853486
843486
833486
823486
813486
803486
793486
783486
773486
763486
753486
743486
733486
723486
7134

11945

In [0]:
import pickle 
filehandler = open("model.pickle", 'wb') 
pickle.dump(model, filehandler)


NameError: ignored

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 3118M    0 10.6M    0     0   499k      0  1:46:30  0:00:21  1:46:09  200k^C


In [0]:
#from google.colab import files
#files.download('occ.npy')

KeyboardInterrupt: ignored

In [0]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19BwWAjwMopAYYYHJmx5Ny4EZmmFxnu9v' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19BwWAjwMopAYYYHJmx5Ny4EZmmFxnu9v" -O model.pickle && rm -rf /tmp/cookies.txt

--2020-05-06 17:08:21--  https://docs.google.com/uc?export=download&confirm=35Lk&id=19BwWAjwMopAYYYHJmx5Ny4EZmmFxnu9v
Resolving docs.google.com (docs.google.com)... 64.233.189.102, 64.233.189.138, 64.233.189.100, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-bs-docs.googleusercontent.com/docs/securesc/451517ostnth8lnspl2d7pbf5cg399ou/b6lac3um8tnnskooghtt445bss0mcdf5/1588784850000/11375003261469243618/10117562806681436359Z/19BwWAjwMopAYYYHJmx5Ny4EZmmFxnu9v?e=download [following]
--2020-05-06 17:08:22--  https://doc-00-bs-docs.googleusercontent.com/docs/securesc/451517ostnth8lnspl2d7pbf5cg399ou/b6lac3um8tnnskooghtt445bss0mcdf5/1588784850000/11375003261469243618/10117562806681436359Z/19BwWAjwMopAYYYHJmx5Ny4EZmmFxnu9v?e=download
Resolving doc-00-bs-docs.googleusercontent.com (doc-00-bs-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting

In [0]:
import pickle 
import glove
filehandler = open("/content/drive/My Drive/model.pickle", 'rb') 
model = pickle.load(filehandler)


/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.6/dist-packages/glove/glove_inner.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [0]:
# Test case 
word0 = m_words['min']
word1 = m_words['din']
word2 = m_words['hest']
word3 = m_words['vores']

vectors = np.zeros((50,4))
vectors[:,0] = model.W[word0]
vectors[:,1] = model.W[word1]
vectors[:,2] = model.W[word2]
vectors[:,3] = model.W[word3]

CO = np.dot(vectors.T, vectors)
square_mag = np.diag(CO)
inv_square_mag = 1 / square_mag
inv_square_mag[np.isinf(inv_square_mag)] = 0
inv_mag = np.sqrt(inv_square_mag)
cosine = CO * inv_mag
cosine = cosine.T * inv_mag
indices = np.argsort(cosine.sum(axis=0))
outlier = indices[0]

print(outlier)

2


In [0]:
from google.colab import files
uploaded = files.upload()

Saving Finn_data_dansk.txt to Finn_data_dansk.txt


In [0]:
four_words_txt = []
with open('Finn_data_dansk.txt', newline='', encoding = 'utf8') as file:
    data = file.read().strip()
lines = data.split("\n")
four_words_txt = [line.split(",") for line in lines]
four_words_txt = [[element.strip() for element in row] for row in four_words_txt]

four_words_txt.pop(0)

['word1', 'word2', 'word3', 'word4']

In [0]:
four_words_txt

[['æble', 'pære', 'kirsebær', 'stol'],
 ['stol', 'bord', 'reol', 'græs'],
 ['græs', 'træ', 'blomst', 'bil'],
 ['bil', 'cykel', 'tog', 'vind'],
 ['vind', 'regn', 'solskin', 'mandag'],
 ['mandag', 'tirsdag', 'søndag', 'tømrer'],
 ['tømrer', 'vvs-mand', 'snedker', 'barn'],
 ['barn', 'far', 'mormor', 'lampe'],
 ['lampe', 'stearinlys', 'lommelygte', 'jern'],
 ['jern', 'guld', 'magnesium', 'sjov'],
 ['sjov', 'dårlig', 'vanvittig', 'papir'],
 ['papir', 'ringbind', 'blyant', 'vagt'],
 ['vagt', 'politimand', 'fængselsbetjent', 'by'],
 ['by', 'landsby', 'købstad', 'småkage'],
 ['småkage', 'citronmåne', 'kringle', 'dør'],
 ['dør', 'væg', 'vindue', 'klaver'],
 ['klaver', 'trompet', 'blokfløjte', 'fandens'],
 ['fandens', 'fuck', 'sgu', 'vand'],
 ['vand', 'jord', 'ild', 'hukommelse'],
 ['hukommelse', 'intelligens', 'emotion', 'Niels Bohr'],
 ['Niels Bohr', 'H.C. Ørsted', 'Ole Rømer', 'Lars Løkke Rasmussen'],
 ['Lars Løkke Rasmussen',
  'Poul Nyrup Rasmussen',
  'Anders Fogh Rasmussen',
  'Peter Schm

In [0]:
cosine_outliers = []
words_not_in_model = 0
for w in four_words_txt: 
  continueBool = False
  try:
    word0 = m_words[w[0]]
    word1 = m_words[w[1]]
    word2 = m_words[w[2]]
    word3 = m_words[w[3]]

    vectors = np.zeros((4, 50))
    vectors[0,:] = model.W[word0].T
    vectors[1,:] = model.W[word1].T
    vectors[2,:] = model.W[word2].T
    vectors[3,:] = model.W[word3].T
  except: 
    continueBool = True
    words_not_in_model +=1
    cosine_outliers.append(str(words_not_in_model ))
  if continueBool: 
       continue


  CO = np.dot(vectors, vectors.T)
  square_mag = np.diag(CO)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = CO * inv_mag
  cosine = cosine.T * inv_mag
  indices = np.argsort(cosine.sum(axis=0))
  outlier = w[indices[0]]
  print(w,outlier)
  cosine_outliers.append(outlier)
    

['æble', 'pære', 'kirsebær', 'stol'] stol
['stol', 'bord', 'reol', 'græs'] reol
['græs', 'træ', 'blomst', 'bil'] bil
['bil', 'cykel', 'tog', 'vind'] tog
['vind', 'regn', 'solskin', 'mandag'] solskin
['mandag', 'tirsdag', 'søndag', 'tømrer'] tømrer
['barn', 'far', 'mormor', 'lampe'] lampe
['lampe', 'stearinlys', 'lommelygte', 'jern'] lommelygte
['jern', 'guld', 'magnesium', 'sjov'] sjov
['sjov', 'dårlig', 'vanvittig', 'papir'] vanvittig
['papir', 'ringbind', 'blyant', 'vagt'] vagt
['vagt', 'politimand', 'fængselsbetjent', 'by'] by
['by', 'landsby', 'købstad', 'småkage'] småkage
['småkage', 'citronmåne', 'kringle', 'dør'] citronmåne
['dør', 'væg', 'vindue', 'klaver'] klaver
['klaver', 'trompet', 'blokfløjte', 'fandens'] fandens
['fandens', 'fuck', 'sgu', 'vand'] vand
['vand', 'jord', 'ild', 'hukommelse'] hukommelse
['bold', 'fjerbold', 'puck', 'mave'] fjerbold
['mave', 'bryst', 'ryg', 'hat'] hat
['hat', 'kasket', 'hue', 'ishockey'] ishockey
['ishockey', 'skiløb', 'skihop', 'fodbold'] fod

In [0]:
sum_cosine = 0
for i in range(100):
    if four_words_txt[i][3] == cosine_outliers[i]:
        sum_cosine += 1
    
print('Outlier detection using cosine', sum_cosine/len(four_words_txt))

Outlier detection using cosine 0.49


In [0]:
print(len(four_words_txt)) 
print(len(cosine_outliers))

100
100


In [0]:
from google.colab import files
uploaded = files.upload()

Saving ProperNounsDanskNy.csv to ProperNounsDanskNy.csv


In [0]:
import pandas as pd
# This is the propernouns
propernouns_data = pd.read_csv("ProperNounsDanskNy.csv",delimiter = ",", encoding = "utf-8")

propernouns = []
for i in range(1, len(propernouns_data)):
    propernouns.append([str(propernouns_data['Word'][i]),str(propernouns_data['Definition'][i])])

In [0]:
propernouns

[['Niels Bohr', 'dansk fysiker'],
 ['H.C. Ørsted', 'dansk udmærkelse'],
 ['Ole Rømer', 'dansk astronom'],
 ['Lars Løkke Rasmussen', 'dansk politiker'],
 ['Lars Løkke Rasmussen', 'dansk politiker'],
 ['Poul Nyrup Rasmussen', 'dansk politiker'],
 ['Anders Fogh Rasmussen',
  'generalsekretær i NATO. Tidligere statsminister i Danmark. Tidligere formand for Venstre'],
 ['Peter Schmeichel', 'dansk fodboldspiller'],
 ['Peter Schmeichel', 'dansk fodboldspiller'],
 ['Kasper Schmeichel', 'dansk fodboldmålmand'],
 ['Brian Laudrup', 'dansk fodboldspiller'],
 ['Caroline Wozniacki', 'dansk tennisspiller'],
 ['Caroline Wozniacki', 'dansk tennisspiller'],
 ['Steffi Graf', 'tysk tennisspiller'],
 ['Serena Williams', 'amerikansk tennisspiller'],
 ['Monaco', 'europæisk mikrostat ved Middelhavet'],
 ['Monaco', 'europæisk mikrostat ved Middelhavet'],
 ['Paris', 'Frankrigs hovedstad'],
 ['Milano', 'major city in Italyengelsk'],
 ['Pia', 'kvindenavn'],
 ['Pia', 'kvindenavn'],
 ['Lone', 'pigenavn'],
 ['Marian

TypeError: ignored

In [0]:
cosine_outliers = []
words_not_in_model = 0
for word_list in four_words_txt: 
  continueBool = False
  try:
    vectors = np.zeros((4, 50))
    if word_list[0][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[0]: 
          definition = propernouns[i][1].split()
          for p in definition: 
            word0 = m_words[p]
            vectors[0,:] += model.W[word0].T
      vectors[0,:] = vectors[0,:]/len(definition)
    else:  
      word0 = m_words[word_list[0]]
      vectors[0,:] = model.W[word0].T


    if word_list[1][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[1]: 
          definition = propernouns[i][1].split()
          for p in definition: 
            word1 = m_words[p]
            vectors[1,:] += model.W[word1].T
      vectors[1,:] = vectors[1,:]/len(definition)
    else:  
      word1 = m_words[word_list[1]]
      vectors[1,:] = model.W[word1].T

    if word_list[2][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[2]: 
          definition = propernouns[i][1].split()
          for p in definition: 
            word2 = m_words[p]
            vectors[2,:] += model.W[word2].T
      vectors[2,:] = vectors[2,:]/len(definition)
    else:  
      word2 = m_words[word_list[2]]
      vectors[2,:] = model.W[word2].T

    if word_list[3][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[3]: 
          definition = propernouns[i][1].split()
          for p in definition: 
            word3 = m_words[p]
            vectors[3,:] += model.W[word3].T
      vectors[3,:] = vectors[3,:]/len(definition)
    else:  
      word3 = m_words[word_list[3]]
      vectors[3,:] = model.W[word3].T
  except: 
    continueBool = True
    words_not_in_model +=1
    cosine_outliers.append(str(words_not_in_model ))
  if continueBool: 
       continue


  CO = np.dot(vectors, vectors.T)
  square_mag = np.diag(CO)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = CO * inv_mag
  cosine = cosine.T * inv_mag
  indices = np.argsort(cosine.sum(axis=0))
  outlier = word_list[indices[0]]
  #print(word_list,outlier)
  cosine_outliers.append(outlier)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide


In [0]:
sum_cosine = 0
for i in range(100):
    if four_words_txt[i][3] == cosine_outliers[i]:
        sum_cosine += 1
    
print('Outlier detection using cosine', sum_cosine/len(four_words_txt))

Outlier detection using cosine 0.51


In [0]:
cosine_outliers = []
words_not_in_model = 0
for word_list in four_words_txt: 
  continueBool = False
  try:
    vectors = np.zeros((4, 50))
    if word_list[0][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[0]: 
          definition = propernouns[i][1].split()
          word0 = m_words[definition[0]]
          vectors[0,:] += model.W[word0].T
      vectors[0,:] = vectors[0,:]/len(definition)
    else:  
      word0 = m_words[word_list[0]]
      vectors[0,:] = model.W[word0].T


    if word_list[1][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[1]: 
          definition = propernouns[i][1].split()
          word1 = m_words[definition[0]]
          vectors[1,:] += model.W[word1].T
      vectors[1,:] = vectors[1,:]/len(definition)
    else:  
      word1 = m_words[word_list[1]]
      vectors[1,:] = model.W[word1].T

    if word_list[2][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[2]: 
          definition = propernouns[i][1].split()
          word2 = m_words[definition[0]]
          vectors[2,:] += model.W[word2].T
      vectors[2,:] = vectors[2,:]/len(definition)
    else:  
      word2 = m_words[word_list[2]]
      vectors[2,:] = model.W[word2].T

    if word_list[3][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[3]: 
          definition = propernouns[i][1].split()
          word3 = m_words[definition[0]]
          vectors[3,:] += model.W[word3].T
      vectors[3,:] = vectors[3,:]/len(definition)
    else:  
      word3 = m_words[word_list[3]]
      vectors[3,:] = model.W[word3].T
  except: 
    continueBool = True
    words_not_in_model +=1
    cosine_outliers.append(str(words_not_in_model ))
  if continueBool: 
       continue


  CO = np.dot(vectors, vectors.T)
  square_mag = np.diag(CO)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = CO * inv_mag
  cosine = cosine.T * inv_mag
  indices = np.argsort(cosine.sum(axis=0))
  outlier = word_list[indices[0]]
  #print(word_list,outlier)
  cosine_outliers.append(outlier)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: divide by zero encountered in true_divide


In [0]:
sum_cosine = 0
for i in range(100):
    if four_words_txt[i][3] == cosine_outliers[i]:
        sum_cosine += 1
    
print('Outlier detection using cosine', sum_cosine/len(four_words_txt))

Outlier detection using cosine 0.55


In [0]:
cosine_outliers = []
words_not_in_model = 0
for word_list in four_words_txt: 
  continueBool = False
  try:
    vectors = np.zeros((4, 50))
    if word_list[0][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[0]: 
          definition = propernouns[i][1].split()
          word0 = m_words[definition[0]]
          vectors[0,:] += model.W[word0].T
      vectors[0,:] = vectors[0,:]/len(definition)
    else:  
      word0 = m_words[word_list[0]]
      vectors[0,:] = model.W[word0].T
  except: 
    vectors[0,:] = np.ones((1,50))*100000
  
  try: 
    if word_list[1][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[1]: 
          definition = propernouns[i][1].split()
          word1 = m_words[definition[0]]
          vectors[1,:] += model.W[word1].T
      vectors[1,:] = vectors[1,:]/len(definition)
    else:  
      word1 = m_words[word_list[1]]
      vectors[1,:] = model.W[word1].T
  except: 
      vectors[1,:] = np.ones((1,50))*100000

  try: 
    if word_list[2][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[2]: 
          definition = propernouns[i][1].split()
          word2 = m_words[definition[0]]
          vectors[2,:] += model.W[word2].T
      vectors[2,:] = vectors[2,:]/len(definition)
    else:  
      word2 = m_words[word_list[2]]
      vectors[2,:] = model.W[word2].T
  except: 
    vectors[2,:] = np.ones((1,50))*100000

  try: 
    if word_list[3][0].isupper():
      for i in range(len(propernouns)): 
        if propernouns[i][0] == word_list[3]: 
          definition = propernouns[i][1].split()
          word3 = m_words[definition[0]]
          vectors[3,:] += model.W[word3].T
      vectors[3,:] = vectors[3,:]/len(definition)
    else:  
      word3 = m_words[word_list[3]]
      vectors[3,:] = model.W[word3].T
  except: 
    vectors[3,:] = np.ones((1,50))*100000
    print('except 3')
 


  CO = np.dot(vectors, vectors.T)
  square_mag = np.diag(CO)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = CO * inv_mag
  cosine = cosine.T * inv_mag
  indices = np.argsort(cosine.sum(axis=0))
  outlier = word_list[indices[0]]
  #print(word_list,outlier)
  cosine_outliers.append(outlier)

except 3
except 3
except 3
except 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in true_divide


In [0]:
sum_cosine = 0
for i in range(100):
    if four_words_txt[i][3] == cosine_outliers[i]:
        sum_cosine += 1
    
print('Outlier detection using cosine', sum_cosine/100)

Outlier detection using cosine 0.55
